In [22]:
# standard libraries
import numpy as np
import pandas as pd
import random
import os
from joblib import Parallel, delayed
import librosa
from sklearn.model_selection import train_test_split
# for normalization & avgpooling features
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, UpSampling2D, Input, TimeDistributed, Dropout, BatchNormalization, Dense
from tensorflow.keras.models import Model
# from tensorflow_addons.optimizers import RectifiedAdam
# import tensorflow_hub as hub
# from tensorflow.keras.layers import TFSMLayer
from vit_keras import vit

# from IPython.display import Audio
import time

In [ ]:

# !pip install tensorflow-addons==0.21.0
# !pip install tensorflow==2.11


# !pip install tensorflow-gpu==2.10
# !pip uninstall tensorflow -y
# !pip install tensorflow==2.15
!pip install tf-models-official==2.15
!pip install keras==2.15
!pip install tensorflow-addons==0.23.0
!pip install vit-keras
# !pip install vit-tensorflow

# !pip uninstall tensorflow -y
# !pip install tensorflow==2.15

# !pip install tensorflow-gpu
# !pip uninstall tensorflow tensorflow-gpu -y
# !pip install tensorflow
# !pip install --upgrade pip setuptools

# print("TensorFlow version:", tf.__version__)
# print("Keras version:", tf.keras.__version__)

# !pip install vit-keras
# !pip install --upgrade tensorflow==2.2
# !pip install --upgrade tensorflow tensorflow-addons
# !pip uninstall keras
# !pip install --upgrade keras==2.14.0
# !pip install --upgrade vit-keras

In [ ]:
import tensorflow_hub as hub
# ViT = tf.keras.Sequential([
#     hub.KerasLayer("https://www.kaggle.com/models/spsayakpaul/vision-transformer/TensorFlow2/vit-r50-l32-fe/1", trainable=True)
# ])

In [ ]:
import psutil

def get_memory_info():
    mem = psutil.virtual_memory()
    total = mem.total / (1024 ** 3)
    available = mem.available / (1024 ** 3)
    print(f"Total memory: {total:.2f} GB")
    print(f"Available memory: {available:.2f} GB")

get_memory_info()

In [ ]:
df = pd.read_csv('/kaggle/input/birdclef-2024/train_metadata.csv')
print(len(df))
print(sorted(df['primary_label'].value_counts().tolist(), reverse=True))

In [6]:
filtered_df = pd.read_csv('/kaggle/input/birdclef-2024/train_metadata.csv')
# filtered_df = import_data('/kaggle/input/birdclef-2024/train_metadata.csv')
# interested_species = ['grewar3', 'asikoe2', 'commyn', 'rorpar'] #'blrwar1', 'woosan', 'grnsan'
# filtered_df = filtered_df[filtered_df['primary_label'].isin(interested_species)]

def remove_nonexistent_files(df, column_name, directory):
    def file_exists(filename):
        return os.path.isfile(os.path.join(directory, filename))
    file_existence_mask = df[column_name].apply(file_exists)
    df_filtered = df[file_existence_mask].copy()
    return df_filtered

column_name = "filename"
directory = "/kaggle/input/birdclef-2024/train_audio"
filtered_df = remove_nonexistent_files(filtered_df, column_name, directory)
# change the file path to your local path
path = '/kaggle/input/birdclef-2024/train_audio/'
sr = 16000

def get_audio_duration(file_path, sr=sr):
    audio, _ = librosa.load(file_path, sr=sr)
    duration = librosa.get_duration(y=audio, sr=sr)
    return duration

In [7]:
%%time
filtered_df.loc[:, 'duration'] = filtered_df['filename'].apply(lambda x: get_audio_duration(path + x))

CPU times: user 24min 3s, sys: 39.6 s, total: 24min 43s
Wall time: 28min 32s


In [8]:
filtered_df = filtered_df.drop_duplicates(subset=['duration', 'latitude', 'type', 'primary_label', 'author'], keep='first')
filtered_df = filtered_df.sample(frac=1).reset_index(drop=True)

print(prev_len:= len(filtered_df))
filtered_df_ = filtered_df[filtered_df['duration']<=245]
print(len(filtered_df_))
print(len(filtered_df_)/prev_len)
print(filtered_df_['duration'].sum()/filtered_df['duration'].sum())

24355
24032
0.9867378361732704
0.8093861384861282


In [9]:
# Assign value to a new column based on the rating column
print(prev_len:= len(filtered_df))
filtered_df['rating_category'] = filtered_df['rating'].apply(lambda x: 'good' if x >= 3.0 else 'bad')
filtered_df = filtered_df[filtered_df['rating_category']=='good']
print(len(filtered_df))
print(len(filtered_df)/prev_len)

24355
21421
0.8795319236296448


In [3]:
def extract_audio(filename, sr=16000):
    filepath = '/kaggle/input/birdclef-2024/train_audio/' + filename
    audio, _ = librosa.load(filepath, sr=sr)
#     audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
#     audio = audio.astype(np.float16)
    return audio.astype(np.float16)

# def pad_array_to_multiple_of_40000(arr):
#     length = len(arr)
#     padding_amount = 40000 - (lenghth%40000)
#     if padding_amount > 0:
#         padded_array = np.pad(arr, (0, padding_amount), mode='constant', constant_values=0)
#     else:
#         padded_array = arr
#     return padded_array.astype(np.float16)

# def pad_audio(audio):
#     desired_length=245*16000
#     current_length = len(audio)
#     if current_length < desired_length:
#         # Pad with zeros if the audio is shorter than the desired length
#         padding_length = desired_length - current_length
#         audio = np.pad(audio, (0, padding_length), mode='constant')
#     else:
#         # Trim the audio if it is longer than the desired length
#         audio = audio[:desired_length]
#     return audio.astype(np.float16)

def pad_array(array):
    target_dim = 98
    current_dim = array.shape[0]
    if current_dim < target_dim:
        padding_amount = target_dim - current_dim
#         padding_width = [(0, padding_amount)] + [(0, 0)] * (array.ndim - 1)
#         padded_array = np.pad(array, padding_width, mode='constant', constant_values=0)
        padding_shape = (padding_amount,) + array.shape[1:]
        padding_array = np.zeros(padding_shape, dtype=array.dtype)
        padded_array = np.concatenate((array, padding_array), axis=0)
        if len(padded_array.shape)==1: print(array.shape, padding_array.shape)
        return padded_array.astype(np.float16)
    else:
        truncated_array = array[:target_dim, ...]
        return truncated_array.astype(np.float16)

def augment_data(audio_array, sr, n_fft):
    audio_array = librosa.effects.pitch_shift(y=audio_array.astype(np.float32), sr=sr,
                                              n_steps=np.random.uniform(low=.970, high=1.03),
                                              n_fft=n_fft)
    audio_array = librosa.effects.time_stretch(y=audio_array,
                                               rate=np.random.uniform(low=.975, high=1.025)) #n_fft=16384
    noise_amp = np.random.normal(loc=0.0, scale=0.005)*np.amax(audio_array)
    audio_array = audio_array + noise_amp*np.random.normal(size=audio_array.shape[0])
    shift_range = int(np.random.uniform(low=-5, high=5)*sr*10)
    audio_array = np.roll(audio_array, shift_range)
    return audio_array.astype(np.float16)

# Function to generate augmented data
def generate_augmented_data(df, column, label_column, threshold, sr=16000, n_fft=2048):
    augmented_data = []
    for label, count in df[label_column].value_counts().items():
        if count < threshold:
            # Select rows with this label
            num_needed = threshold-count
            label_df = df[df[label_column] == label]
            num_samples = min(num_needed, len(label_df))
            label_df = label_df.sample(n= num_samples,
                                        random_state=random.randint(0, 10000))

            for _, row in label_df.iterrows():
                npy_array = row[column]
                augmented_npy = augment_data(npy_array, sr, n_fft)
                augmented_row = row.copy()
                augmented_row[column] = augmented_npy
                augmented_row['augmented'] = True
                augmented_data.append(augmented_row)
    # Append the augmented data to the original dataframe
    augmented_df = pd.DataFrame(augmented_data)
    result_df = pd.concat([df, augmented_df], ignore_index=True)
    return result_df

def trim_df_by_label(df, label_column, threshold):
    # Create an empty DataFrame to store the results
    trimmed_df = pd.DataFrame(columns=df.columns)
    # Group by the label column
    grouped = df.groupby(label_column)
    for label, group in grouped:
        if len(group) > threshold:
            sampled_group = group.sample(n=threshold, random_state=1)
        else:
            sampled_group = group
        
        trimmed_df = pd.concat([trimmed_df, sampled_group])
    return trimmed_df

def extract_framed(audio):
    """
    Load audio files from the given DataFrame, extract framed audios,
    and add the framed audios to the given DataFrame

    Parameters:
      dataframe (pd.DataFrame): DataFrame containing audio file information.
      augment (bool): Whether to apply augmentation to the audio.
    """
    window_size_s=2.5
    hop_size_s=2.5
    sr=16000
    frame_length = int(window_size_s * sr)
    frame_step = int(hop_size_s * sr)
    return tf.cast(tf.signal.frame(audio, frame_length, frame_step, pad_end=True), tf.float16)

# # Function to compute MFCCs from frames
# def compute_mfcc(frames):
#     n_mfcc=33
#     sr=16000
#     hop_length=626
#     frames = frames.numpy().astype(np.float32)
#     mfccs = [librosa.feature.mfcc(y=frame, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length) for frame in frames]
#     return np.array(mfccs).astype(np.float16)
def compute_mfcc(frame, sr, n_mfcc, hop_length):
    return librosa.feature.mfcc(y=frame, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)


# def compute_chroma(frames):
#     sr = 16000
#     hop_length = 626
#     n_chroma = 12  # The number of chroma bins to use
#     frames = frames.numpy().astype(np.float32)
#     chroma_stft = [librosa.feature.chroma_stft(y=frame, sr=sr, hop_length=hop_length, n_chroma=n_chroma) for frame in frames]
#     return np.array(chroma_stft).astype(np.float16)

# def compute_rms(frames):
#     sr = 16000
#     hop_length = 626
#     frames = frames.numpy().astype(np.float32)
#     rms = [librosa.feature.rms(y=frame, hop_length=hop_length) for frame in frames]
#     return np.array(rms).astype(np.float16)
def compute_rms(frame, hop_length):
    return librosa.feature.rms(y=frame, hop_length=hop_length)

def extract_features(row):
#     mfcc = compute_mfcc(row['npy'])
    n_mfcc=33
    sr=16000
    hop_length=626
    frames = row['npy'].numpy().astype(np.float32)
    mfcc = Parallel(n_jobs=-1, backend='threading')(delayed(compute_mfcc)(frame, sr, n_mfcc, hop_length) for frame in frames)
    mfcc = np.array(mfcc).astype(np.float16)
    #     chroma = compute_chroma(row['npy'])
#     rms = compute_rms(row['npy'])
    rms = Parallel(n_jobs=-1, backend='threading')(delayed(compute_rms)(frame, hop_length) for frame in frames)
    rms = np.array(rms).astype(np.float16)
    row['npy'] = None
    return pd.Series({'mfcc': mfcc, 'rms': rms})

# Function to normalize the entire column data using Min-Max Normalization
def normalize_column(data, scaler):
#     print(data[208])
    stacked = np.concatenate(data)
    original_shape = stacked.shape
#     print(len(data), data[0].shape, original_shape)
    reshaped_stacked = stacked.reshape(-1, original_shape[-1])
    normalized_reshaped = scaler.fit_transform(reshaped_stacked)
    normalized_stacked = normalized_reshaped.reshape(original_shape).astype(np.float16)
    split_indices = np.cumsum([arr.shape[0] for arr in data[:-1]])
    normalized_data = np.split(normalized_stacked, split_indices)
#     print(len(normalized_data), normalized_data[0].shape)
    return normalized_data

# Function to concatenate numpy arrays along a specified axis and replace original values with None
def concat_and_replace(row, axis=0):
    reshaped_long = np.full(row['rms'].shape, row['longitude'])
    reshaped_lat = np.full(row['rms'].shape, row['latitude'])
    concatenated_data = np.concatenate((row['rms'], reshaped_long), axis=-2)
    concatenated_data = np.concatenate((concatenated_data, reshaped_lat), axis=-2)
    concatenated_data = np.concatenate((row['mfcc'], concatenated_data), axis=-2)
    concatenated_data = np.transpose(concatenated_data, (0, 2, 1))
    concatenated_data = np.expand_dims(concatenated_data, axis=-1)
#     print(concatenated_data.shape)
    # Replace original column values with None to preserve space
    row['rms'] = None
    row['mfcc'] = None
    row['longitude'] = None
    row['latitude'] = None    
    return concatenated_data.astype(np.float16)

def birds_stratified_split(df, target_col, test_size=0.2):
    class_counts = df[target_col].value_counts()
    low_count_classes = class_counts[class_counts < 2].index.tolist() ### Birds with single counts

    df['train'] = df[target_col].isin(low_count_classes)

    train_df, test_df = train_test_split(df[~df['train']], test_size=test_size, stratify=df[~df['train']][target_col], random_state=42)

    train_df = pd.concat([train_df, df[df['train']]], axis=0).reset_index(drop=True)

    # Remove the 'valid' column
    train_df.drop('train', axis=1, inplace=True)
    test_df.drop('train', axis=1, inplace=True)

    return train_df, test_df

In [10]:
print(len(filtered_df))
# print(new_df['primary_label'].value_counts())
# print(type(new_df["primary_label"].iloc[0]), type(new_df["features"].iloc[0]))

21421


In [27]:
filtered_df = filtered_df.sample(frac=1).reset_index(drop=True)
interested_species = ['grewar3', 'asikoe2', 'commyn', 'rorpar'] #'blrwar1', 'woosan', 'grnsan'
filtered_df = filtered_df[filtered_df['primary_label'].isin(interested_species)]
new_df = filtered_df[['primary_label', 'latitude', 'longitude', 'filename']].head(20)
print(f"num of species: {new_df['primary_label'].nunique()}")
# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False)
# Fit and transform the labels
_ = encoder.fit_transform(new_df['primary_label'].values.reshape(-1, 1))

long_lat_scaler = MinMaxScaler(feature_range=(-1, 1))
_ = long_lat_scaler.fit_transform(np.concatenate((new_df['longitude'].values, new_df['latitude'].values), axis=-1).reshape(-1, 1))

new_df['npy'] = new_df['filename'].apply(extract_audio)
new_df['augmented'] = False
# threshold = min(max(new_df['primary_label'].value_counts().values.tolist()), 4)
threshold = 4
print(f"Threshold: {threshold}")
while min(new_df['primary_label'].value_counts().values.tolist())<threshold:
    new_df = generate_augmented_data(new_df, 'npy', 'primary_label', threshold, sr=16000, n_fft=2048)
new_df = trim_df_by_label(new_df, 'primary_label', threshold)
# new_df['npy'] = new_df['npy'].apply(pad_audio)
# Feature Extraction
new_df['npy'] = new_df['npy'].apply(extract_framed)
new_df[['mfcc', 'rms']] = new_df.apply(lambda row: extract_features(row), axis=1)
new_df['mfcc'] = normalize_column(new_df['mfcc'].values, MinMaxScaler(feature_range=(-1, 1)))
new_df['rms'] = normalize_column(new_df['rms'].values, MinMaxScaler(feature_range=(-1, 1)))
new_df['longitude'] = long_lat_scaler.transform(new_df['longitude'].values.reshape(-1, 1)).astype(np.float16)
new_df['latitude'] = long_lat_scaler.transform(new_df['latitude'].values.reshape(-1, 1)).astype(np.float16)
new_df['features'] = new_df.apply(lambda row: concat_and_replace(row, axis=1), axis=1)
new_df['features'] = new_df['features'].apply(pad_array)
new_df = new_df[['primary_label', 'features']]
print(len(new_df))
display(new_df.head(5))

train_df, test_df = birds_stratified_split(new_df, 'primary_label', test_size=0.3)
# Fit and transform the labels
train_labels = encoder.transform(train_df['primary_label'].values.reshape(-1, 1)).astype(np.float16)
test_labels = encoder.transform(test_df['primary_label'].values.reshape(-1, 1)).astype(np.float16)
# Convert features to numpy arrays
train_features = np.stack(train_df['features'].values)
test_features = np.stack(test_df['features'].values)

num of species: 4


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Threshold: 4


/tmp/ipykernel_34/230960520.py:92: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trimmed_df = pd.concat([trimmed_df, sampled_group])


16


,primary_label,features
18,asikoe2,"[[[[-0.379], [0.75], [0.75], [0.75], [0.75], [..."
8,asikoe2,"[[[[-1.], [0.7295], [0.732], [0.7417], [0.741]..."
6,asikoe2,"[[[[-0.287], [0.9307], [0.734], [0.798], [0.78..."
4,asikoe2,"[[[[-0.872], [0.789], [0.758], [0.7583], [0.77..."
1,commyn,"[[[[0.1997], [0.8857], [0.6846], [0.7563], [0...."


In [28]:
import vit_keras
library_path = os.path.dirname(vit_keras.__file__)
print("Library path:", library_path)
!cp -r {library_path} /kaggle/working/vit_keras

# Add the extracted directory to PYTHONPATH
# import sys
# sys.path.insert(0, '/kaggle/working/vit_keras')

Library path: /opt/conda/lib/python3.10/site-packages/vit_keras


In [26]:
# Define the ViT layer using TensorFlow SavedModel
# class TFSMLayer(tf.keras.layers.Layer):
#     def __init__(self, model_path, call_endpoint='serving_default', **kwargs):
#         super(TFSMLayer, self).__init__(**kwargs)
#         self.model = tf.saved_model.load(model_path)
#         self.call_fn = self.model.signatures[call_endpoint]
#         self.trainable = True
#     def call(self, inputs):
#         return self.call_fn(inputs)

# ViT = TFSMLayer("/kaggle/input/vision-transformer/tensorflow2/vit-b8-fe/1")
# ViT.trainable = True  # Set the model to be trainable

# ViT = TFSMLayer("/kaggle/input/vision-transformer/tensorflow2/vit-b8-fe/1", call_endpoint='serving_default')
# ViT.trainable = True

# ViT_input = tf.keras.layers.Input(shape=(224, 224, 3))
# ViT = vision_transformer = hub.KerasLayer("https://www.kaggle.com/models/spsayakpaul/vision-transformer/TensorFlow2/vit-s16-fe/1", trainable=True)(ViT_input)

ViT = vit.vit_b16(
    image_size=224,
    activation='sigmoid',
    pretrained=True,
    include_top=True,
    pretrained_top=True,
    classes=182
)

input_shape = (98, 64, 36, 1)
inputs = Input(shape=(input_shape))
batch_size = 20
x = TimeDistributed(Conv2D(30, (9, 9), padding='valid', activation='relu', strides=(2, 1)))(inputs)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = TimeDistributed(Conv2D(6, (13, 13), padding='valid', activation='tanh', strides=(1, 1)))(x)
x = Dropout(0.2)(x)
x = tf.keras.layers.Reshape((224, 224, 3))(x)
# x = ViT(x)['output_0']
print(ViT, type(ViT))
print(x, type(x))
x = ViT(x)
print(x.shape)
x = tf.keras.layers.Dense(384, activation = 'relu')(x)
x = BatchNormalization()(x)
# x = Dense(182, activation = 'softmax')(x)
x = Dense(79, activation = 'softmax')(x)
model = Model(inputs=inputs, outputs=x)
model.summary()

/opt/conda/lib/python3.10/site-packages/vit_keras/vit.py:139: UserWarning: Can only use pretrained_top with imagenet21k+imagenet2012 if classes = 1000. Setting manually.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


<keras.src.engine.functional.Functional object at 0x7eeed4531b70> <class 'keras.src.engine.functional.Functional'>
KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), name='reshape_3/Reshape:0', description="created by layer 'reshape_3'") <class 'keras.src.engine.keras_tensor.KerasTensor'>
(None, 1000)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 98, 64, 36, 1)]   0         
                                                                 
 time_distributed_2 (TimeDi  (None, 98, 28, 28, 30)    2460      
 stributed)                                                      
                                                                 
 dropout_2 (Dropout)         (None, 98, 28, 28, 30)    0         
                                                                 
 batch_normalization_2 (Bat  (None, 98, 28, 28,

In [27]:
# Save the model after each chunk
output_dir = '/kaggle/working/'
model_save_path = os.path.join(output_dir, 'bird_clef_24_model_79.h5')
model.save(model_save_path)

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
learning_rate = 1e-3
# optimizer = tfa.optimizers.RectifiedAdam(lr=learning_rate)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping_callbacks = tf.keras.callbacks.EarlyStopping(patience = 20, restore_best_weights = True, verbose = 1)

history = model.fit(train_features, train_labels, epochs=40, batch_size=32, validation_data=(test_features, test_labels), callbacks = early_stopping_callbacks)

In [13]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# List available devices
device_list = tf.config.list_physical_devices('GPU')
print(device_list)

# if device_list:
#     tf.config.experimental.set_memory_growth(device_list[0], True)

Num GPUs Available:  2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [25]:
# filtered_df.to_csv("/kaggle/working/filtered_df.csv", index=False)
# filtered_df = pd.read_csv('/kaggle/input/filtered-df/filtered_df.csv')
# filtered_df.to_csv('/kaggle/working/filtered_df.csv', index=False)
print(filtered_df['primary_label'].nunique(), len(filtered_df))
threshold = 80
value_counts = filtered_df['primary_label'].value_counts()
values_to_keep = value_counts[value_counts >= threshold].index
filtered_df_ = filtered_df[filtered_df['primary_label'].isin(values_to_keep)]

print(filtered_df_['primary_label'].nunique(), len(filtered_df_))
# Step 3: Save the Filtered DataFrame
# filtered_df_.to_csv('/kaggle/working/filtered_df_80.csv', index=False)


182 21421
79 18295


In [ ]:
filtered_df = filtered_df.sample(frac=1).reset_index(drop=True)
df = filtered_df[['primary_label', 'latitude', 'longitude', 'filename']]
print(f"num of species: {df['primary_label'].nunique()}")
initial_length = len(df)
label_counts = df['primary_label'].value_counts().to_dict()
new_df = pd.DataFrame(columns=[df.columns])

learning_rate = 1e-4
# optimizer = tfa.optimizers.RectifiedAdam(lr=learning_rate)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping_callbacks = tf.keras.callbacks.EarlyStopping(patience = 20, restore_best_weights = True, verbose = 1)

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False)
# Fit and transform the labels
_ = encoder.fit_transform(df['primary_label'].values.reshape(-1, 1))
# test_labels = encoder.transform(test_df[['primary_label']])
# Initialize the MinMaxScaler with feature range (-1, 1)
long_lat_scaler = MinMaxScaler(feature_range=(-1, 1))
_ = long_lat_scaler.fit_transform(df['longitude'].values.reshape(-1, 1))

# Path to save models
output_dir = '/kaggle/working/saved_models/'
os.makedirs(output_dir, exist_ok=True)

train_count = 0
# While current DataFrame length is greater than a certain value
while len(df) > initial_length * 0.005:  # For example, continue until half of the initial length
    start_time = time.time()  # Record start time
    print(f"trained data count: {train_count}")
    print(f"progress through data: {(train_count/initial_length)*100}%")
    new_df = df.sample(frac=1).reset_index(drop=True)
    new_df = pd.DataFrame(columns=df.columns)
    # Iterate through each key in the stored frequency dictionary
    for label, count in label_counts.items():
        # Extract 2% of the data with that value
        sample_size = max(int(count * 0.02), 1)
        extracted_data = df[df['primary_label'] == label].head(sample_size)
        df = df.drop(extracted_data.index)
        new_df = pd.concat([new_df, extracted_data])
    # Keep count of trained data
    train_count += len(new_df)
    # Load, Augment, Pad and Trim Audio Data
    new_df['npy'] = new_df['filename'].apply(extract_audio)
    print(f"extracted npy")
    new_df['augmented'] = False
    threshold = min(max(new_df['primary_label'].value_counts().values.tolist()), 8)
    # threshold = 20
    print(f"Threshold: {threshold}")
    while min(new_df['primary_label'].value_counts().values.tolist())<threshold:
        new_df = generate_augmented_data(new_df, 'npy', 'primary_label', threshold, sr=16000, n_fft=2048)
    new_df = trim_df_by_label(new_df, 'primary_label', threshold)
    # new_df['npy'] = new_df['npy'].apply(pad_audio)
    # Feature Extraction
    new_df['npy'] = new_df['npy'].apply(extract_framed)
    new_df[['mfcc', 'rms']] = new_df.apply(lambda row: extract_features(row), axis=1)
    new_df['mfcc'] = new_df['mfcc'].apply(pad_array)
    new_df['rms'] = new_df['rms'].apply(pad_array)
    new_df['mfcc'] = normalize_column(new_df['mfcc'].values, MinMaxScaler(feature_range=(-1, 1)))
    new_df['rms'] = normalize_column(new_df['rms'].values, MinMaxScaler(feature_range=(-1, 1)))
    new_df['longitude'] = long_lat_scaler.transform(new_df['longitude'].values.reshape(-1, 1)).astype(np.float16)
    new_df['latitude'] = long_lat_scaler.transform(new_df['latitude'].values.reshape(-1, 1)).astype(np.float16)
    new_df['features'] = new_df.apply(lambda row: concat_and_replace(row, axis=1), axis=1)
    new_df['features'] = new_df['features'].apply(pad_array)
    new_df = new_df[['primary_label', 'features']]
    print(f"ready to train")
    # Split Stratified Train and Test Sets
    print(len(new_df))
    train_df, test_df = birds_stratified_split(new_df, 'primary_label', 0.3)
    # Fit and transform the labels
    train_labels = encoder.transform(train_df['primary_label'].values.reshape(-1, 1)).astype(np.float16)
    test_labels = encoder.transform(test_df['primary_label'].values.reshape(-1, 1)).astype(np.float16)
    # Convert features to numpy arrays
    train_features = np.stack(train_df['features'].values)
    test_features = np.stack(test_df['features'].values)
    train_features = np.nan_to_num(train_features, nan=0)
    test_features = np.nan_to_num(test_features, nan=0)
    # Train The Model
    history = model.fit(train_features, train_labels, epochs=40, batch_size=4, validation_data=(test_features, test_labels), callbacks = early_stopping_callbacks)
    
    # Save the model after each chunk
    model_save_path = os.path.join(output_dir, 'bird_clef_24_model.h5')
    model.save(model_save_path)
    print(f'Saved model to {model_save_path}')
    end_time = time.time()  # Record end time
    print(f'Time taken for prev iteration: {(end_time - start_time)/60} mins')

In [ ]:
filtered_df = filtered_df.sample(frac=1).reset_index(drop=True)

######################################################################
interested_species = ['grewar3', 'asikoe2', 'commyn', 'rorpar'] #'blrwar1', 'woosan', 'grnsan'
filtered_df = filtered_df[filtered_df['primary_label'].isin(interested_species)]

df = filtered_df[['primary_label', 'latitude', 'longitude', 'filename']]
print(f"num of species: {df['primary_label'].nunique()}")
initial_length = len(df)
label_counts = df['primary_label'].value_counts().to_dict()
new_df = pd.DataFrame(columns=[df.columns])

learning_rate = 1e-4
# optimizer = tfa.optimizers.RectifiedAdam(lr=learning_rate)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping_callbacks = tf.keras.callbacks.EarlyStopping(patience = 20, restore_best_weights = True, verbose = 1)

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False)
# Fit and transform the labels
_ = encoder.fit_transform(df['primary_label'].values.reshape(-1, 1))
# test_labels = encoder.transform(test_df[['primary_label']])
# Initialize the MinMaxScaler with feature range (-1, 1)
long_lat_scaler = MinMaxScaler(feature_range=(-1, 1))
_ = long_lat_scaler.fit_transform(df['longitude'].values.reshape(-1, 1))

# Path to save models
output_dir = '/kaggle/working/saved_models/'
os.makedirs(output_dir, exist_ok=True)

train_count = 0
# While current DataFrame length is greater than a certain value
while len(df) > initial_length * 0.005:  # For example, continue until half of the initial length
    start_time = time.time()  # Record start time
    print(f"trained data count: {train_count}")
    print(f"progress through data: {(train_count/initial_length)*100}%")
    new_df = df.sample(frac=1).reset_index(drop=True)
    new_df = pd.DataFrame(columns=df.columns)
    # Iterate through each key in the stored frequency dictionary
    for label, count in label_counts.items():
        # Extract 2% of the data with that value
        sample_size = max(int(count * 0.02), 1)
        extracted_data = df[df['primary_label'] == label].head(sample_size)
        df = df.drop(extracted_data.index)
        new_df = pd.concat([new_df, extracted_data])
    # Keep count of trained data
    train_count += len(new_df)
    # Load, Augment, Pad and Trim Audio Data
    new_df['npy'] = new_df['filename'].apply(extract_audio)
    print(f"extracted npy")
    new_df['augmented'] = False
    threshold = min(max(new_df['primary_label'].value_counts().values.tolist()), 8)
    # threshold = 20
    print(f"Threshold: {threshold}")
    while min(new_df['primary_label'].value_counts().values.tolist())<threshold:
        new_df = generate_augmented_data(new_df, 'npy', 'primary_label', threshold, sr=16000, n_fft=2048)
    new_df = trim_df_by_label(new_df, 'primary_label', threshold)
    print(f"augmented npy")
#     new_df['npy'] = new_df['npy'].apply(pad_audio)
    # Feature Extraction
#     new_df['npy'] = new_df['npy'].apply(pad_array_to_multiple_of_40000)
    new_df['npy'] = new_df['npy'].apply(extract_framed)
    new_df[['mfcc', 'rms']] = new_df.apply(lambda row: extract_features(row), axis=1)
    
    new_df['mfcc'] = new_df['mfcc'].apply(pad_array)
    new_df['rms'] = new_df['rms'].apply(pad_array)
    print(f"extracted mfcc and rms")
    new_df['mfcc'] = normalize_column(new_df['mfcc'].values, MinMaxScaler(feature_range=(-1, 1)))
    new_df['rms'] = normalize_column(new_df['rms'].values, MinMaxScaler(feature_range=(-1, 1)))
    new_df['longitude'] = long_lat_scaler.transform(new_df['longitude'].values.reshape(-1, 1)).astype(np.float16)
    new_df['latitude'] = long_lat_scaler.transform(new_df['latitude'].values.reshape(-1, 1)).astype(np.float16)
    new_df['features'] = new_df.apply(lambda row: concat_and_replace(row, axis=1), axis=1)
    new_df = new_df[['primary_label', 'features']]
    print(f"ready to train")
    # Split Stratified Train and Test Sets
    print(len(new_df))
    train_df, test_df = birds_stratified_split(new_df, 'primary_label', 0.3)
    # Fit and transform the labels
    train_labels = encoder.transform(train_df['primary_label'].values.reshape(-1, 1)).astype(np.float16)
    test_labels = encoder.transform(test_df['primary_label'].values.reshape(-1, 1)).astype(np.float16)
    # Convert features to numpy arrays
    train_features = np.stack(train_df['features'].values)
    test_features = np.stack(test_df['features'].values)
    train_features = np.nan_to_num(train_features, nan=0)
    test_features = np.nan_to_num(test_features, nan=0)
    # Train The Model
    history = model.fit(train_features, train_labels, epochs=40, batch_size=28, validation_data=(test_features, test_labels), callbacks = early_stopping_callbacks)
    
    # Save the model after each chunk
    model_save_path = os.path.join(output_dir, 'bird_clef_24_model.h5')
    model.save(model_save_path)
    print(f'Saved model to {model_save_path}')
    end_time = time.time()  # Record end time
    print(f'Time taken for prev iteration: {(end_time - start_time)/60} mins')

In [22]:
# print(np.isnan(train_features).sum())
# print(np.isnan(test_features).sum())
print(train_features.size, np.isnan(train_features).sum())
print(test_features.size, np.isnan(test_features).sum())

# Remove rows with NaN values from train features and labels
# train_mask = ~np.isnan(train_features).any(axis=0)
train_features_ = np.nan_to_num(train_features, nan=0)
test_features_ = np.nan_to_num(test_features, nan=0)
# train_labels_ = train_labels[train_mask]
print(np.isnan(train_features_).sum(), np.isnan(test_features_).sum())
print(train_features_.size, test_features_.size)

# # Remove rows with NaN values from test features and labels
# test_mask = ~np.isnan(test_features).any()
# test_features_ = test_features[test_mask]
# test_labels_ = test_labels[test_mask]

# print(np.isnan(train_features_).sum())
# print(np.isnan(test_features_).sum())

230082048 188160
98671104 62720
0 0
230082048 98671104


In [36]:
print(new_df['mfcc'][new_df['mfcc']==None])
# for row in new_df['mfcc'].values:
#     if len(row.shape)!=3: print(row.shape)
file_list = []
for index in new_df.index:
    row = new_df.loc[index]
    if len(row['mfcc'].shape)!=3:
        print(row.shape)
        file_list.append(f"/kaggle/input/birdclef-2024/train_audio/{row['filename']}")
print(file_list)

Series([], Name: mfcc, dtype: object)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
['/kaggle/input/birdclef-2024/train_audio/darter2/XC684291.ogg', '/kaggle/input/birdclef-2024/train_audio/darter2/XC684291.ogg', '/kaggle/input/birdclef-2024/train_audio/darter2/XC684291.ogg', '/kaggle/input/birdclef-2024/train_audio/darter2/XC684291.ogg', '/kaggle/input/birdclef-2024/train_audio/greegr/XC116543.ogg', '/kaggle/input/birdclef-2024/train_audio/greegr/XC116543.ogg', '/kaggle/input/birdclef-2024/train_audio/lesyel1/XC151547.ogg', '/kaggle/input/birdclef-2024/train_audio/lesyel1/XC151547.ogg', '/kaggle/input/birdclef-2024/train_audio/lesyel1/XC151547.ogg', '/kaggle/input/birdclef-2024/train_audio/lesyel1/XC151547.ogg', '/kaggle/input/birdclef-2024/train_audio/rerswa1/XC423900.ogg', '/kaggle/input/birdclef-2024/train_audio/rerswa1/XC423900.ogg', '/kaggle/input/birdclef-2024/train_audio/rerswa1/XC423900.ogg']


In [42]:
audio, _ = librosa.load(file_list[5], sr=16000)
Audio(audio, rate=16000)

In [ ]:
%%time
# train_df['npy'] = train_df['filename'].apply(extract_audio)
test_df['npy'] = test_df['filename'].apply(extract_audio)
# train_df['augmented'] = False
test_df['augmented'] = False
# train_df = generate_augmented_data(train_df, 'npy', 'primary_label', 270, sr=16000, n_fft=2048)
while min(test_df['primary_label'].value_counts().values.tolist())<115:
    test_df = generate_augmented_data(test_df, 'npy', 'primary_label', 115, sr=16000, n_fft=2048)
# train_df = trim_df_by_label(train_df, 'primary_label', 270)
test_df = trim_df_by_label(test_df, 'primary_label', 115)
# train_df['npy'] = train_df['npy'].apply(pad_audio)
test_df['npy'] = test_df['npy'].apply(pad_audio)

In [ ]:
def extract_framed(audio):
    """
    Load audio files from the given DataFrame, extract framed audios,
    and add the framed audios to the given DataFrame

    Parameters:
      dataframe (pd.DataFrame): DataFrame containing audio file information.
      augment (bool): Whether to apply augmentation to the audio.
    """
    window_size_s=2.5
    hop_size_s=2.5
    sr=16000
    frame_length = int(window_size_s * sr)
    frame_step = int(hop_size_s * sr)
    return tf.cast(tf.signal.frame(audio, frame_length, frame_step, pad_end=False), tf.float16)

In [ ]:
%%time
# train_df['npy'] = train_df['npy'].apply(extract_framed)
test_df['npy'] = test_df['npy'].apply(extract_framed)

In [ ]:
# Function to compute MFCCs from frames
def compute_mfcc(frames):
    n_mfcc=32
    sr=16000
    hop_length=655
    frames = frames.numpy().astype(np.float32)
    mfccs = [librosa.feature.mfcc(y=frame, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length) for frame in frames]
    return np.array(mfccs).astype(np.float16)

# def compute_chroma(frames):
#     sr = 16000
#     hop_length = 625
#     n_chroma = 12  # The number of chroma bins to use
#     frames = frames.numpy().astype(np.float32)
#     chroma_stft = [librosa.feature.chroma_stft(y=frame, sr=sr, hop_length=hop_length, n_chroma=n_chroma) for frame in frames]
#     return np.array(chroma_stft).astype(np.float16)

def compute_rms(frames):
    sr = 16000
    hop_length = 655
    frames = frames.numpy().astype(np.float32)
    rms = [librosa.feature.rms(y=frame, hop_length=hop_length) for frame in frames]
    return np.array(rms).astype(np.float16)

def extract_features(row):
    mfcc = compute_mfcc(row['npy'])
#     chroma = compute_chroma(row['npy'])
    rms = compute_rms(row['npy'])
    row['npy'] = None
    return pd.Series({'mfcc': mfcc, 'rms': rms})

In [ ]:
%%time
# test_df['npy'] = test_df['npy'].apply(lambda x: compute_mfcc(x))
# # Rename the column to "mfcc"
# test_df.rename(columns={'npy': 'mfcc'}, inplace=True)

# Apply the function to each row
test_df[['mfcc', 'rms']] = test_df.apply(lambda row: extract_features(row), axis=1)

In [ ]:
# Function to normalize the entire column data using Min-Max Normalization
def normalize_column(data, scaler):
    stacked = np.concatenate(data)
    original_shape = stacked.shape
    reshaped_stacked = stacked.reshape(-1, original_shape[-1])
    normalized_reshaped = scaler.fit_transform(reshaped_stacked)
    normalized_stacked = normalized_reshaped.reshape(original_shape)
    split_indices = np.cumsum([arr.shape[0] for arr in data[:-1]])
    normalized_data = np.split(normalized_stacked, split_indices)
    return normalized_data

# Function to concatenate numpy arrays along a specified axis and replace original values with None
def concat_and_replace(row, axis=0):
#     concatenated_data = np.concatenate([row['mfcc'], row['rms']], axis=axis)
    reshaped_long = np.full(row['rms'].shape[:2], row['longitude'])
    reshaped_lat = np.full(row['rms'].shape[:2], row['latitude'])
    concatenated_data = np.concatenate((row['rms'], reshaped_long[..., np.newaxis]), axis=-1)
    concatenated_data = np.concatenate((concatenated_data, reshaped_lat[..., np.newaxis]), axis=-1)
    concatenated_data = np.concatenate((rows['mfcc'][..., np.newaxis], concatenated_data[..., np.newaxis]), axis=-1)
    # Replace original column values with None
    row['A'] = None
    row['B'] = None
    row['C'] = None
    # Rearrange the array to shape (98, 64, 32, 3)
    concatenated_data = np.transpose(concatenated_data, (0, 2, 1, 3))
    return concatenated_data

In [ ]:
# reshaped_long = np.full(test_df['mfcc'].iloc[0].shape[:2], test_df['longitude'].iloc[0])
# concatenated_data = np.concatenate((test_df['mfcc'].iloc[0], reshaped_long[..., np.newaxis]), axis=-1)
# print(reshaped_long.shape)
# print(concatenated_data.shape)

In [ ]:
# Initialize the MinMaxScaler with feature range (-1, 1)
scaler = MinMaxScaler(feature_range=(-1, 1))

test_df['mfcc'] = normalize_column(test_df['mfcc'], scaler)
test_df['rms'] = normalize_column(test_df['rms'], scaler)
test_df['longitude'] = scaler.fit_transform(df[['longitude']])
test_df['latitude'] = scaler.fit_transform(df[['latitude']])

# Apply the function to each row
test_df['features'] = test_df.apply(lambda row: concat_and_replace(row, axis=1), axis=1)

In [ ]:
train_df, test_df = birds_stratified_split(filtered_df, 'primary_label', 0.3)
display(len(train_df))
display(len(test_df))

In [ ]:
# Apply the function to each row
test_df['features'] = test_df.apply(lambda row: concat_and_replace(row, axis=1), axis=1)

In [ ]:
# Convert the MFCC data to TensorFlow Tensors
test_df['mfcc'] = test_df['mfcc'].apply(lambda x: tf.convert_to_tensor(x, dtype=tf.float16))

In [ ]:
%%time
# Function to invert MFCCs back to audio signal
def mfcc_to_audio(mfcc, sr=16000):
    # Inverse MFCC to Mel Spectrogram
    mel_spec = librosa.feature.inverse.mfcc_to_mel(mfcc)
    # Inverse Mel Spectrogram to audio signal
    audio = librosa.feature.inverse.mel_to_audio(mel_spec, sr=sr, hop_length=655)
    return audio

# Convert the first instance of MFCC back to audio for integrity test
# mfcc_sample = test_df[test_df['augmented']==True]['mfcc'].iloc[3].numpy().astype(np.float32)  # Convert back to float32 for inversion
mfcc_sample = test_df[test_df['augmented']==True]['mfcc'].iloc[3].astype(np.float32) 
mfcc_concat = np.concatenate(mfcc_sample, axis=1)
audio_signal = mfcc_to_audio(mfcc_concat)
Audio(audio_signal, rate=16000)

In [ ]:
for mfcc in test_df.head(10)['mfcc']: print(mfcc.shape)

In [ ]:
# model_handle = "/kaggle/input/vision-transformer/tensorflow2/vit-b8-fe/1"
# ViT = hub.KerasLayer(model_handle, trainable=True)

In [ ]:
# ViT_handle = "/kaggle/input/vision-transformer/tensorflow2/vit-b8-fe/1"
# ViT = hub.KerasLayer(ViT_handle, trainable=True)
# config = ViT.get_config()
# ViT = tf.keras.Sequential.from_config([config])

# model = tf.keras.Sequential([
#         Input(shape=((98, 64, 32, 3))),
#         TimeDistributed(Conv2D(6, (17, 17), padding='valid', activation='relu', strides=(3, 1))),
#         Dropout(0.3),
#         tf.keras.layers.Reshape((224, 224, 3)),
#         tf.keras.layers.BatchNormalization(),
#         BatchNormalization(),
#         ViT
#     ],
#     name = 'CNN_vision_transformer')

# model.summary()

In [ ]:
# # Define a custom layer to wrap the tf.image.resize function
# from tensorflow.keras.layers import Layer
# class ViT_Layer(Layer):
#     def __init__(self, size, **kwargs):
#         super(ViT_Layer, self).__init__(**kwargs)
#         self.size = size

#     def call(self, inputs):
#         return ViT(inputs, self.size)



In [ ]:
# Convert features to numpy arrays
train_features = np.stack(train_df['features'].values)
test_features = np.stack(test_df['features'].values)

In [ ]:
# ViT_handle = "/kaggle/input/vision-transformer/tensorflow2/vit-b8-fe/1"
# ViT = hub.KerasLayer(ViT_handle, trainable=True)
# Load the saved model

ViT = TFSMLayer("/kaggle/input/vision-transformer/tensorflow2/vit-b8-fe/1", call_endpoint='serving_default')
ViT.trainable = True
input_shape = (98, 64, 32, 3)
inputs = Input(shape=(input_shape))
batch_size = 20
x = TimeDistributed(Conv2D(30, (9, 5), padding='valid', activation='relu', strides=(2, 1)))(inputs)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = TimeDistributed(Conv2D(6, (13, 13), padding='valid', activation='tanh', strides=(1, 1)))(x)
x = Dropout(0.2)(x)
x = tf.keras.layers.Reshape((224, 224, 3))(x)
x = ViT(x)['output_0']
x = tf.keras.layers.Dense(384, activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dense(128, activation = 'softmax')(x)
model = Model(inputs=inputs, outputs=x)
model.summary()

In [ ]:
learning_rate = 1e-3  # Example learning rate, you may need to tune this
optimizer = tfa.optimizers.RectifiedAdam(lr=learning_rate)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping_callbacks = tf.keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True, verbose = 1)

In [ ]:
# Train the model
history = model.fit(train_features, train_labels, epochs=50, batch_size=32, validation_data=(test_features, test_labels), callbacks = early_stopping_callbacks)
